# Set up

In [22]:
# import libraries 
import pandas as pd
import numpy as np
import simfin as sf
import datetime
%matplotlib inline

#set the local directory where data-files are stored
#change to your own directory
sf.set_data_dir('C:/Users/think/Desktop/UVA/2020Spring/STAT_4996_Capstone/simfin_data')

# 一个cell显示所有output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

# Set up API key
sf.set_api_key(api_key='free')

#display all columns
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [23]:
%qtconsole

# Define Function

In [24]:
#calculate the proportion of non-missing value in each column
def prop_nonmissing(df):
    df2 = df.reset_index().copy()
    output = pd.DataFrame(index = df2.columns)
    nomiss_prop = []
    for c in df2.columns:
        nomiss_prop.append(round((1-(df2.loc[:,c].isnull().sum()/len(df2.loc[:,c])))*100,2))
    output['Prop'] = nomiss_prop
    return output

# Load in data 

In [25]:
#load in entire annual income statement data
df_income_a = sf.load_income(variant='annual', market='us').reset_index()
#load in entire annual balance sheet data 
df_balance_a = sf.load_balance(variant='annual', market='us').reset_index()
#load in entire annual balance sheet data 
df_cashflow_a = sf.load_cashflow(variant='annual', market='us').reset_index()
#load in companies detail
comp = sf.load_companies(market='us').reset_index()
industry = sf.load_industries().reset_index()

Dataset "us-income-annual" on disk (9 days old).
- Loading from disk ... Done!
Dataset "us-balance-annual" on disk (9 days old).
- Loading from disk ... Done!
Dataset "us-cashflow-annual" on disk (9 days old).
- Loading from disk ... Done!
Dataset "us-companies" on disk (1 days old).
- Loading from disk ... Done!
Dataset "industries" on disk (9 days old).
- Loading from disk ... Done!


# Merging Dtat

In [26]:
comp.head(2)
industry.head(2)
comp.shape
industry.shape

,Ticker,SimFinId,Company Name,IndustryId
0,A,45846,AGILENT TECHNOLOGIES INC,106001.0
1,AA,367153,Alcoa Corp,110004.0


,IndustryId,Sector,Industry
0,100001,Industrials,Industrial Products
1,100002,Industrials,Business Services


(2067, 4)

(71, 3)

In [27]:
len(set(df_cashflow_a['Ticker']) & set(df_income_a['Ticker']))
len(set(df_cashflow_a['Ticker']) & set(df_balance_a['Ticker']))
#三个表cover的公司完全相同

1915

1915

In [28]:
#left join industry on company
comp['IndustryId'] = comp[['IndustryId']].astype('float')
df = pd.merge(comp, industry, on ='IndustryId', how = 'left',suffixes=('','_right'))
df.head(5)
df.shape

,Ticker,SimFinId,Company Name,IndustryId,Sector,Industry
0,A,45846,AGILENT TECHNOLOGIES INC,106001.0,Healthcare,Medical Diagnostics & Research
1,AA,367153,Alcoa Corp,110004.0,Basic Materials,Metals & Mining
2,AAC,939324,"AAC Holdings, Inc.",NaN,NaN,NaN
3,AAL,68568,American Airlines Group Inc.,100006.0,Industrials,Airlines
4,AAME,450021,ATLANTIC AMERICAN CORP,104004.0,Financial Services,Insurance - Life


(2067, 6)

In [29]:
#left join income statement on previous df 
df2 = pd.merge(df,df_income_a,on = ['Ticker','SimFinId'], how = 'right')

'''
#check if joined properly: missing value percentage match
prop_nonmissing(df2).sort_values(by = 'Prop',ascending = False)
df2.head(2)
df2.columns
df2.shape
'''

"\n#check if joined properly: missing value percentage match\nprop_nonmissing(df2).sort_values(by = 'Prop',ascending = False)\ndf2.head(2)\ndf2.columns\ndf2.shape\n"

In [30]:
#left join balance sheet on previous df 
df_balance_a.shape
df3 = pd.merge(df2,df_balance_a,on = ['Ticker','SimFinId','Fiscal Year','Currency'], how = 'left', suffixes=('','_b'))

'''
#take a look at the merge data
df3.head(2)
df3.columns
#check if merge successfully, if successful, the row number should not change
df2.shape
df3.shape
prop_nonmissing(df3.iloc[:,~df3.columns.isin(df2.columns)]).sort_values(by = 'Prop', ascending = False)
'''

(14768, 29)

"\n#take a look at the merge data\ndf3.head(2)\ndf3.columns\n#check if merge successfully, if successful, the row number should not change\ndf2.shape\ndf3.shape\nprop_nonmissing(df3.iloc[:,~df3.columns.isin(df2.columns)]).sort_values(by = 'Prop', ascending = False)\n"

In [31]:
#left join balance sheet on previous df 
df_cashflow_a.shape
df4 = pd.merge(df3,df_cashflow_a,on = ['Ticker','SimFinId','Fiscal Year','Currency'], how = 'left', suffixes=('','_c'))

'''
#take a look at the merge data
df4.head(2)
df4.columns
#check if merge successfully, if successful, the row number should not change
df3.shape
df4.shape
prop_nonmissing(df4.iloc[:,~df4.columns.isin(df3.columns)]).sort_values(by = 'Prop', ascending = False)
'''

(14768, 27)

"\n#take a look at the merge data\ndf4.head(2)\ndf4.columns\n#check if merge successfully, if successful, the row number should not change\ndf3.shape\ndf4.shape\nprop_nonmissing(df4.iloc[:,~df4.columns.isin(df3.columns)]).sort_values(by = 'Prop', ascending = False)\n"

In [33]:
#delete old dataframe
del df3
del df2
del df
del df_balance_a
del df_cashflow_a
del df_income_a

In [66]:
#check overlapping columns 
df4.filter(regex='_').head()
df4[['Report Date','Fiscal Period','Publish Date','Shares (Basic)','Shares (Diluted)','Depreciation & Amortization']].head()
#check if we can delete any overlapping column 
#can delete report date_b and _c
#can delete depreciation & amortizaton 
sum((df4['Report Date_b'] == df4['Report Date_c'])== False)
sum((df4['Publish Date_b'] == df4['Publish Date_c'])== False)
sum((df4['Shares (Basic)_c'] == df4['Shares (Basic)'])== False)
sum((df4['Depreciation & Amortization'] == df4['Depreciation & Amortization_c'])== False)


,Fiscal Period_b,Publish Date_b,Shares (Basic)_b,Shares (Diluted)_b,Fiscal Period_c,Publish Date_c,Shares (Basic)_c,Shares (Diluted)_c
0,Q4,2009-10-05,357000000.0,359000000.0,FY,2009-10-05,363000000.0,371000000.0
1,Q4,2009-12-21,343000000.0,343000000.0,FY,2009-12-21,346000000.0,346000000.0
2,Q4,2010-12-20,344000000.0,356000000.0,FY,2010-12-20,347000000.0,353000000.0
3,Q4,2011-12-16,346000000.0,353000000.0,FY,2011-12-16,347000000.0,355000000.0
4,Q4,2012-12-20,348000000.0,353000000.0,FY,2012-12-20,348000000.0,353000000.0


,Report Date,Fiscal Period,Publish Date,Shares (Basic),Shares (Diluted),Depreciation & Amortization
0,2008-10-31,FY,2009-10-05,363000000.0,371000000.0,201000000.0
1,2009-10-31,FY,2009-12-21,346000000.0,346000000.0,162000000.0
2,2010-10-31,FY,2010-12-20,347000000.0,353000000.0,202000000.0
3,2011-10-31,FY,2011-12-16,347000000.0,355000000.0,253000000.0
4,2012-10-31,FY,2012-12-20,348000000.0,353000000.0,301000000.0


KeyError: 'Report Date_b'

In [36]:
#rearrange data frame
#drop useless columns 
df4.drop(columns=['Report Date_b','Report Date_c','Depreciation & Amortization'],inplace=True)
df4.columns

Index(['Ticker', 'SimFinId', 'Company Name', 'IndustryId', 'Sector',
       'Industry', 'Report Date', 'Currency', 'Fiscal Year', 'Fiscal Period',
       'Publish Date', 'Shares (Basic)', 'Shares (Diluted)', 'Revenue',
       'Cost of Revenue', 'Gross Profit', 'Operating Expenses',
       'Selling, General & Administrative', 'Research & Development',
       'Operating Income (Loss)', 'Non-Operating Income (Loss)',
       'Interest Expense, Net', 'Pretax Income (Loss), Adj.',
       'Abnormal Gains (Losses)', 'Pretax Income (Loss)',
       'Income Tax (Expense) Benefit, Net',
       'Income (Loss) from Continuing Operations',
       'Net Extraordinary Gains (Losses)', 'Net Income', 'Net Income (Common)',
       'Fiscal Period_b', 'Publish Date_b', 'Shares (Basic)_b',
       'Shares (Diluted)_b', 'Cash, Cash Equivalents & Short Term Investments',
       'Accounts & Notes Receivable', 'Inventories', 'Total Current Assets',
       'Property, Plant & Equipment, Net',
       'Long Term Inves

In [37]:
df4.rename(columns={"Depreciation & Amortization_c": "Depreciation & Amortization"},inplace=True)
df4.columns

Index(['Ticker', 'SimFinId', 'Company Name', 'IndustryId', 'Sector',
       'Industry', 'Report Date', 'Currency', 'Fiscal Year', 'Fiscal Period',
       'Publish Date', 'Shares (Basic)', 'Shares (Diluted)', 'Revenue',
       'Cost of Revenue', 'Gross Profit', 'Operating Expenses',
       'Selling, General & Administrative', 'Research & Development',
       'Operating Income (Loss)', 'Non-Operating Income (Loss)',
       'Interest Expense, Net', 'Pretax Income (Loss), Adj.',
       'Abnormal Gains (Losses)', 'Pretax Income (Loss)',
       'Income Tax (Expense) Benefit, Net',
       'Income (Loss) from Continuing Operations',
       'Net Extraordinary Gains (Losses)', 'Net Income', 'Net Income (Common)',
       'Fiscal Period_b', 'Publish Date_b', 'Shares (Basic)_b',
       'Shares (Diluted)_b', 'Cash, Cash Equivalents & Short Term Investments',
       'Accounts & Notes Receivable', 'Inventories', 'Total Current Assets',
       'Property, Plant & Equipment, Net',
       'Long Term Inves

# Checking for missing values for each finantial ratios

以下所有的formula已全部替换为表内对应字段名
注意
1. Stock Price 目前没有对应字段
2. 有一些标注（year-start)的字段，对应的字段名内没有“(year-start)”


| Liquidity Ratios     | Formula |
| :---- | :---- |
| Current ratio |  Total Current Assets/Total Current Liabilities       |
| Quick ratio | (Cash, Cash Equivalents & Short Term Investments+ Accounts & Notes Receivable)/Total Current Liabilities|
|Net working capital to assets ratio| (Total Current Assets-Total Current Liabilities)/Total Current Assets|
|Cash ratio| Cash, Cash Equivalents & Short Term Investments+ Accounts/Total Current Liabilities|


| Leverage Ratios     | Formula |
| :---- | :---- |
| Long-term debt ratio |  Long Term Debt/(Long Term Debt+ Total Equity) |
| Liabilities to Equity Ratio|Total Liabilities/ Total Equity)|
|Total Debt ratio|Total Liabilities/Total Assets|
| Debt to Assets Ratio|(Long Term Debt+Short Term Debt)/Total Assets|
|Interest coverage ratio|Operating Income (Loss)/Interest Expense, Net|
|Cash coverage ratio|(Operating Income (Loss)+Depreciation & Amortization)/Interest Expense, Net|


| Profitability Ratios     | Formula |
| :---- | :---- |
|After Tax Operating Income|Revenue-Operating Expenses-Depreciation & Amortization-Income Tax (Expense) Benefit, Net|
| Return on Asset |After Tax Operating Income/Total Assets  |
| Return on Capital|After Tax Operating Income/(Long Term Debt+ Total Equity)|
| Return on Equity| After Tax Operating Income/Total Equity|
| Economic Value Added| After Tax Operating Income-Cost of capital（need outside data）* (Long Term Debt+ Total Equity)|




| Efficiency Ratios     | Formula |
| :---- | :---- |
| Operating profit margin| Operating Income (Loss)/Revenue|
|Net Profit Margin| Net Income (Common)/Revenue|
|Asset Turnover|Revenue/Total Assets(year-start)|
|Receivable turover|Revenue/Accounts & Notes Receivable(year-start)|
|Inventory turnover|Cost of Revenue/Inventories(year-start)|



| Performance Ratios     | Formula |
| :---- | :---- |
| Market Value added| Stock Price * Shares (Basic) -Total Equity|
| Market to Book ratio|Stock Price * Shares (Basic)/Total Equity|
|Earning per Share|Net Income (Common)/Shares (Basic)|
|Sales per Share|Revenue/Shares (Basic)|
|Price to Earning Ratio|Stock Price/Earning Per Share|
|Price to Book Ratio|Stock Price/(Total Equity/Shares (Basic))|



In [67]:
#只选取所需的column
cols = ['Ticker', 'SimFinId', 'Company Name', 'IndustryId', 'Sector',
       'Industry', 'Report Date', 'Fiscal Year','Publish Date', 'Shares (Basic)',
        'Revenue','Cost of Revenue',  'Operating Expenses',
       'Operating Income (Loss)', 'Interest Expense, Net', 
       'Income Tax (Expense) Benefit, Net',
       'Income (Loss) from Continuing Operations', 'Net Income (Common)',
       'Fiscal Period_b', 'Publish Date_b', 'Shares (Basic)_b',
       'Shares (Diluted)_b', 'Cash, Cash Equivalents & Short Term Investments',
       'Accounts & Notes Receivable', 'Inventories', 'Total Current Assets',
        'Total Assets',  'Total Current Liabilities', 'Long Term Debt',
        'Total Liabilities', 'Total Equity', 
       'Fiscal Period_c', 'Publish Date_c', 'Shares (Basic)_c',
       'Shares (Diluted)_c', 'Net Income/Starting Line',
       'Depreciation & Amortization', 'Net Cash from Operating Activities',
        'Dividends Paid']
df = df4[cols]
prop_nonmissing(df).sort_values(by='Prop', ascending = False)

,Prop
index,100.00
Income (Loss) from Continuing Operations,100.00
Net Cash from Operating Activities,100.00
Publish Date_c,100.00
Fiscal Period_c,100.00
Total Liabilities,100.00
Total Current Liabilities,100.00
Total Assets,100.00
Ticker,100.00
Fiscal Period_b,100.00


In [55]:

#count the number of rows with no missing values. 
df.notnull().all(axis=1).sum() #3985行没missing value
#count the number of companies with no missing value
df[df.notnull().all(axis=1)]['Ticker'].nunique() #582个公司完全没有missing value

3985

582

In [65]:
df[df['Shares (Basic)'].isnull()].head(30)

,Ticker,SimFinId,Company Name,IndustryId,Sector,Industry,Report Date,Fiscal Year,Publish Date,Shares (Basic),Revenue,Cost of Revenue,Operating Expenses,Operating Income (Loss),"Interest Expense, Net","Income Tax (Expense) Benefit, Net",Income (Loss) from Continuing Operations,Net Income (Common),Fiscal Period_b,Publish Date_b,Shares (Basic)_b,Shares (Diluted)_b,"Cash, Cash Equivalents & Short Term Investments",Accounts & Notes Receivable,Inventories,Total Current Assets,Total Assets,Total Current Liabilities,Long Term Debt,Total Liabilities,Total Equity,Fiscal Period_c,Publish Date_c,Shares (Basic)_c,Shares (Diluted)_c,Net Income/Starting Line,Depreciation & Amortization,Net Cash from Operating Activities,Dividends Paid
91,AB,823898,ALLIANCEBERNSTEIN HOLDING L.P.,104001.0,Financial Services,Asset Management,2015-12-31,2015,2016-02-11,NaN,NaN,NaN,NaN,NaN,NaN,-24320000.0,185764000,185764000,Q4,2016-02-11,NaN,NaN,NaN,NaN,NaN,NaN,1576120000,274000,NaN,274000,1.575846e+09,FY,2016-02-11,NaN,NaN,1.857640e+08,NaN,192789000,-192299000.0
92,AB,823898,ALLIANCEBERNSTEIN HOLDING L.P.,104001.0,Financial Services,Asset Management,2016-12-31,2016,2017-02-14,NaN,NaN,NaN,NaN,NaN,NaN,-22803000.0,216586000,216586000,Q4,2017-02-14,NaN,NaN,NaN,NaN,NaN,NaN,1540508000,619000,NaN,619000,1.539889e+09,FY,2017-02-14,NaN,NaN,2.165860e+08,NaN,169531000,-169731000.0
93,AB,823898,ALLIANCEBERNSTEIN HOLDING L.P.,104001.0,Financial Services,Asset Management,2017-12-31,2017,2018-02-13,NaN,NaN,NaN,NaN,NaN,NaN,-24971000.0,207422000,207422000,Q4,2018-02-13,NaN,NaN,NaN,NaN,NaN,NaN,1544704000,1154000,NaN,1154000,1.543550e+09,FY,2018-02-13,NaN,NaN,2.074220e+08,NaN,202410000,-202388000.0
94,AB,823898,ALLIANCEBERNSTEIN HOLDING L.P.,104001.0,Financial Services,Asset Management,2018-12-31,2018,2019-02-13,NaN,NaN,NaN,NaN,NaN,NaN,-28250000.0,242397000,242397000,Q4,2019-02-13,NaN,NaN,NaN,NaN,NaN,NaN,1490701000,644000,NaN,644000,1.490057e+09,FY,2019-02-13,NaN,NaN,2.423970e+08,NaN,279282000,-280722000.0
176,ABTL,445589,Autobytel Inc,101002.0,Technology,Online Media,2010-12-31,2010,2011-08-05,NaN,5.153400e+07,-3.203200e+07,-2.445000e+07,-4.948000e+06,NaN,-182000.0,-8564000,-8564000,Q4,2011-08-05,NaN,NaN,8.819000e+06,9067000.0,NaN,1.908300e+07,37832000,9272000,5.000000e+06,14729000,2.310300e+07,FY,2011-08-05,NaN,NaN,-8.564000e+06,1.365000e+06,-4346000,NaN
224,ACF,748227,"General Motors Financial Company, Inc.",NaN,NaN,NaN,2018-12-31,2018,2019-02-06,NaN,1.401600e+10,NaN,-8.439000e+09,5.577000e+09,-3.225000e+09,-323000000.0,1570000000,1570000000,Q4,2019-02-06,NaN,NaN,4.883000e+09,NaN,NaN,5.812400e+10,109920000000,49586000000,4.815300e+10,98261000000,1.165900e+10,FY,2019-02-06,NaN,NaN,1.570000e+09,7.697000e+09,7372000000,-434000000.0
356,ADPT,118952,Adeptus Health Inc.,NaN,NaN,NaN,2013-12-31,2013,2014-08-08,NaN,1.028830e+08,NaN,-1.019920e+08,8.910000e+05,-2.827000e+06,-720000.0,-2984000,0,Q4,2014-08-08,NaN,NaN,1.149500e+07,15887000.0,1494000.0,3.307100e+07,183292000,25424000,7.884900e+07,104641000,7.865100e+07,FY,2014-08-08,NaN,NaN,-2.984000e+06,7.920000e+06,6872000,NaN
485,AGI,111482,"Affinion Group, Inc.",110004.0,Basic Materials,Metals & Mining,2010-12-31,2010,2011-07-28,NaN,1.376300e+09,-3.758000e+08,-9.361000e+08,6.440000e+07,-1.464000e+08,-12800000.0,-141600000,-142600000,Q4,2011-07-28,NaN,NaN,1.211000e+08,119300000.0,NaN,4.774000e+08,1515200000,560400000,1.685200e+09,2380100000,-8.649000e+08,FY,2011-07-28,NaN,NaN,-1.416000e+08,1.952000e+08,121800000,NaN
486,AGI,111482,"Affinion Group, Inc.",110004.0,Basic Materials,Metals & Mining,2011-12-31,2011,2012-03-01,NaN,1.535200e+09,-4.415000e+08,-1.044000e+09,4.970000e+07,-1.473000e+08,-5200000.0,-108800000,-109700000,Q4,2012-03-01,NaN,NaN,8.630000e+07,126800000.0,NaN,4.403000e+08,1651000000,601200000,1.922500e+09,2662700000,-1.011700e+09,FY,2012-03-01,NaN,NaN,-1.088000e+08,2.387000e+08,107000000,NaN
487,AGI,111482,"Affinion Group, Inc.",110004.0,Basic Materials,Metals & Mining,2012-12-31,2012,2013-02-28,NaN,1.494600e+09,-4.595000e+